# Request filtered company data from Linkedin API
https://pypi.python.org/pypi/python-linkedin/4.0

In [ ]:
import pickle
import os
import json
import requests

import pandas as pd
from sqlalchemy import create_engine

from linkedin import linkedin

In [ ]:
# store -r access_token
access_token = os.getenv('LINKEDIN_ACCESS_TOKEN') # Use token from redirect URI given in `linkedin_auth` notebook
application = linkedin.LinkedInApplication(token=access_token)
# access_token

`application` object can be used to run querries as per, http://ozgur.github.io/python-linkedin/

https://developer.linkedin.com/docs/fields/positions

https://developer.linkedin.com/docs/fields/companies

https://developer.linkedin.com/docs/fields/company-profile

https://developer.linkedin.com/docs/guide/v2/concepts/paging

In [ ]:
rspnse = application.search_company(
        selectors=[{'companies': ['id',
                                  'name',
                                  'universal-name',
                                  {'company-type': 'name'},
                                  'website-url',
                                  {'industries': 'name'},
                                  {'status': 'name'},
                                  'twitter-id',
                                  {'employee-count-range': 'code'},
                                  'specialties',
                                  'description',
                                  'locations',
                                  'founded-year',
                                  'num-followers',
                                 ]
        }], # https://stackoverflow.com/questions/30743910/linkedin-company-search-api-multiple-facets?rq=1
        params={'start': 0, 'count': 20, 'facet': ['company-size,D', # 51 - 200 employees ( https://developer.linkedin.com/docs/fields/company-profile )
                                                  'industry,4',  # Software ( https://developer.linkedin.com/docs/reference/industry-codes )
                                                  'location,us:84' # SF Bay ( https://developer.linkedin.com/docs/ref/v2/standardized-data/locations/regions )
                                                  ]
               }
    )
rspnse

In [ ]:
# Search URL is https://api.linkedin.com/v1/company-search:(companies:(name,universal-name,website-url))?start=0&count=10&keywords=apple%20microsoft
responses = []
# for j in range(0,12000,2000):
i = 820 # 0
while i < 2000:
    response = application.search_company(
        selectors=[{'companies': ['id',
                                  'name',
                                  'universal-name',
                                  {'company-type': 'name'},
                                  'website-url',
                                  {'industries': 'name'},
                                  {'status': 'name'},
                                  'twitter-id',
                                  {'employee-count-range': 'code'},
                                  'specialties',
                                  'description',
                                  'locations',
                                  'founded-year',
                                  'num-followers',
                                 ]
        }],
        params={'start': i, 'count': i + 20, 'facet': ['company-size,D',
                                                  'industry,4',
                                                  'location,us:84'
                                                  ]
               }
    )
    if response['companies']['values']:
        responses.append(response)
    else:
        break
    i += 20

    with open('lkdn_co_software_ba_szD_0-2k_b.dat', 'wb') as f: # +str((j) / 1000)+'-'+str((j + 2000) / 1000)+'k.dat', 'wb') as f:
            pickle.dump(responses, f)

In [ ]:
# companies_df = pd.to_json("lkdn_co_software_ba_szD.json")
# companies_df.to_csv("lkdn_co_software_ba_szD", encoding='utf-8')

In [ ]:
# engine = create_engine('postgresql://uname@localhost:5432/mydatabase')
# companies_df.to_sql('Linkedin_Companies_Sofrware_SF_Bay_Area', engine)